In [11]:
import re
import os
import xml.etree.ElementTree as ET
import mysql.connector 
import pandas as pd

In [12]:
class Datos_xml:

# Esta clase almacenará la función para leer y limpiar los datos xml.
# Como parámetro recibirá el nombre del archivo.

    def __init__(self, nombre_archivo):
        self.nombre_archivo = nombre_archivo

    #Función para leer el archivo xml
    def lectura_xml(self):
    

        tree = ET.parse(self.nombre_archivo)
        root = tree.getroot()

        self.xml_lista = []
        for child in root:
            xml_dict = {}
            for subchild in child:
                xml_dict[subchild.tag] = subchild.text
            self.xml_lista.append(xml_dict) # Es una lista que contiene diccionarios
        return self.xml_lista

    # Esta función actualiza los datos de la columna género. 
    # Como parámetro recibe la variable creada con el método lectura_xml.
    def actualización_género(self,xml_lista):
    
        self.xml_lista = xml_lista
        
        for i in xml_lista:
            
            if i["gender"] == '0':
                i["gender"] = "Man"
            elif i["gender"] == '1':
                i["gender"] = "Woman"
            elif i["gender"] == '2':
                i["gender"] = "Nonbinary"
            elif i["gender"] == '3':
                i["gender"] = "Prefer not to say"
            elif i["gender"] == '4':
                i["gender"] = "Prefer to self-describe"

        return xml_lista

    def limpieza_datos(self,xml_lista):

    # Con esta función se preparan los datos para su inserción:
    # Como argumento recibe la variable creada con el método atualización genero.

        self.xml_lista = xml_lista

        # Eliminar la primera columna
        try:
            for i in xml_lista:
                i.pop("level_0")
                i["index"] = int(i["index"])
        except: pass

        # Convertir los datos de la columna index en tipo integer

        try:
            for i in xml_lista:
                i["index"] = int(i["index"])
        except: pass

        # Aplicar la condición if para evitar problemas con index_sql, se aplica el método setdefault para buscar la key index_sql y asignarle un value de tipo integer.

        try:
            for i in xml_lista:
                if (xml_lista[0] != "index_sql"):
                    i.setdefault("index_sql", int(i["index"]))
        except: pass

        # Convertir los datos en una lista de tuplas.
    
        tupla_xml = []

        for i in xml_lista:
            tupla_xml.append(tuple(i.values()))
    
        return tupla_xml


In [13]:
# Creación de instancia
archivo_xml = Datos_xml ("data_xml.xml")

In [14]:
# Aplicación del método de lectura a la instancia
lectura_xml = archivo_xml.lectura_xml()

In [15]:
# Aplicación del método de actualización de los datos de género a la instancia
genero_xml = archivo_xml.actualización_género(lectura_xml)

In [16]:
# Aplicación del método de limpieza a la instancia, da lugar al archivo limpio con los datos a insertar.
archivo_xml_limpio = archivo_xml.limpieza_datos(genero_xml)

In [17]:
print(archivo_xml_limpio[0])

(1, '784', '50-54', 'Man', 1)


In [18]:
class Insertar_datos_xml:
# Esta clase creará una instancia para la inserción de los datos limpios de los arhivos .xml.
# Como argumento recibirá el nombre del archivo de xml limpio (archivo_xml_limpio).

    def __init__(self, fichero_xml):
        
        self.fichero_xml = fichero_xml

    def insercion_datos(self):
        # Con esta función se insertan los datos limpios del fichero xml wn la tabla_xml de la Base de datos.
        # No requiere parámetro.

        cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                    host='127.0.0.1', database= "proyecto_1_equipo_1")
        cursor = cnx.cursor()
        query_xml = "INSERT INTO data_xml(index_xml, `time`, age, gender, index_sql) VALUES (%s,%s,%s,%s,%s);"
        
        try:
            cursor.execute("SET GLOBAL FOREIGN_KEY_CHECKS=0;") # Se desactiva temporalmente la foreign key
            cursor.executemany (query_xml, self.fichero_xml )
            cnx.commit()
            print(cursor.rowcount, "registros insertados correctamente.")
            cursor.execute("SET GLOBAL FOREIGN_KEY_CHECKS=1;") # Se reactiva la foreign key
        
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
        cursor.close()
        cnx.close()

In [19]:
insercion_xml = Insertar_datos_xml(archivo_xml_limpio)


In [20]:
insercion_xml.insercion_datos()

25972 registros insertados correctamente.
